<h1><center>
Toxic Comments Kaggle Competition Classifying
</center></h1>
<h2><center>
Modeling the Data
</center></h2>
<h3><center>
Marcel Colvin 912033961
</center></h3>

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import time
import re
import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv("training_data.csv")
test = pd.read_csv("test_data.csv")

In [3]:
len(train)
len(test)

153164

# MLP Classifier with w2v algorithm
### This had 95.04% accuracy

In [4]:
train["avg_word"] = train["avg_word"].apply(lambda x: 
                           np.fromstring(
                               x.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' '))

In [5]:
test["avg_word"] = test["avg_word"].apply(lambda x: 
                           np.fromstring(
                               x.replace('\n','')
                                .replace('[','')
                                .replace(']','')
                                .replace('  ',' '), sep=' '))

In [6]:
train_matrix = np.zeros((len(train),300), dtype="float64")
test_matrix = np.zeros((len(test),300), dtype="float64")
for i in range(len(train)):
    train_matrix[i] = train.iloc[i]["avg_word"].reshape((1,300))
for i in range(len(test)):
        test_matrix[i] = test.iloc[i]["avg_word"].reshape((1,300))

In [7]:
y_train = [train["toxic"], train["severe_toxic"], train["obscene"], train["threat"], train["insult"], train["identity_hate"]]

In [8]:
x_train = train["avg_word"].values

In [9]:
model = []
for i in range(6):
    m = MLPClassifier(hidden_layer_sizes=(30,30,30), random_state=5)
    model.append(m)

In [10]:
for i in range(6):
    model[i].partial_fit(train_matrix, y_train[i], [0,1])

In [11]:
result = test[['id']].copy() 

In [12]:
result['toxic'] = model[0].predict_proba(test_matrix)[:,1]
result['severe_toxic'] = model[1].predict_proba(test_matrix)[:,1]
result['obscene'] = model[2].predict_proba(test_matrix)[:,1]
result['threat'] = model[3].predict_proba(test_matrix)[:,1]
result['insult'] = model[4].predict_proba(test_matrix)[:,1]
result['identity_hate'] = model[5].predict_proba(test_matrix)[:,1]

In [13]:
result.to_csv("submission.csv", index = False)

# TFIDF Logistitc Regression, "comment_text" uncleaned
### This had 93.94% accuracy

In [14]:
tfidfvect = TfidfVectorizer(max_features=5000,stop_words='english')

In [15]:
X = tfidfvect.fit_transform(train["comment_text"])

In [16]:
y_train = [train["toxic"], train["severe_toxic"], train["obscene"], train["threat"], train["insult"], train["identity_hate"]]

In [17]:
testX = tfidfvect.transform(test["comment_text"])

In [18]:
model = []
for i in range(6):
    m = LogisticRegression(C=10.0)
    model.append(m)

In [19]:
for i in range(6):
    model[i].fit(X,y_train[i])

In [20]:
result = test[['id']].copy() 

In [21]:
result['toxic'] = model[0].predict_proba(testX)[:,1]
result['severe_toxic'] = model[1].predict_proba(testX)[:,1]
result['obscene'] = model[2].predict_proba(testX)[:,1]
result['threat'] = model[3].predict_proba(testX)[:,1]
result['insult'] = model[4].predict_proba(testX)[:,1]
result['identity_hate'] = model[5].predict_proba(testX)[:,1]

In [22]:
result.to_csv("submission.csv", index = False)

# Cleaning "comment_text" to run TFIDF Logistic Regression Again
### This had 96.30% accuracy

In [23]:
def text_cleaner(word):
    word = word.lower()
    word = re.sub(r"\'s", ' ', word)
    word = re.sub("<[^>]*>", " ", word)
    word = re.sub('\W', ' ', word)
    word = re.sub('\s+', ' ', word)
    word = word.strip(' ')
    return word

In [24]:
train["comment_text"] = train['comment_text'].map(lambda word : text_cleaner(word))
test["comment_text"] = test['comment_text'].map(lambda word : text_cleaner(word))

In [25]:
tfidfvect = TfidfVectorizer(max_features=5000,stop_words='english')

In [26]:
X = tfidfvect.fit_transform(train["comment_text"])

In [27]:
y_train = [train["toxic"], train["severe_toxic"], train["obscene"], train["threat"], train["insult"], train["identity_hate"]]

In [28]:
testX = tfidfvect.transform(test["comment_text"])

In [29]:
model = []
for i in range(6):
    m = LogisticRegression(C=10.0)
    model.append(m)

In [30]:
for i in range(6):
    model[i].fit(X,y_train[i])

In [31]:
result = test[['id']].copy() 

In [32]:
result['toxic'] = model[0].predict_proba(testX)[:,1]
result['severe_toxic'] = model[1].predict_proba(testX)[:,1]
result['obscene'] = model[2].predict_proba(testX)[:,1]
result['threat'] = model[3].predict_proba(testX)[:,1]
result['insult'] = model[4].predict_proba(testX)[:,1]
result['identity_hate'] = model[5].predict_proba(testX)[:,1]

In [33]:
result.to_csv("submission.csv", index = False)

# HHH

In [34]:
import glob
path = '/home/marcel/Github/Toxic_comments/nyt-comments/comments/'
nyt_comments = pd.DataFrame()
for file_name in glob.glob(path+'*.csv'):
    print(file_name)
    nyt_comments = nyt_comments.append(pd.read_csv(file_name)[['commentBody', 'commentID', "reportAbuseFlag", 'userLocation', 'userID']])

/home/marcel/Github/Toxic_comments/nyt-comments/comments/CommentsMarch2018.csv
/home/marcel/Github/Toxic_comments/nyt-comments/comments/CommentsApril2018.csv
/home/marcel/Github/Toxic_comments/nyt-comments/comments/CommentsMarch2017.csv
/home/marcel/Github/Toxic_comments/nyt-comments/comments/CommentsApril2017.csv
/home/marcel/Github/Toxic_comments/nyt-comments/comments/CommentsJan2018.csv
/home/marcel/Github/Toxic_comments/nyt-comments/comments/CommentsMay2017.csv
/home/marcel/Github/Toxic_comments/nyt-comments/comments/CommentsJan2017.csv
/home/marcel/Github/Toxic_comments/nyt-comments/comments/CommentsFeb2018.csv
/home/marcel/Github/Toxic_comments/nyt-comments/comments/CommentsFeb2017.csv


In [35]:
nyt_comments["commentBody"] = nyt_comments['commentBody'].map(lambda word : text_cleaner(word))

In [36]:
nyt_comments = nyt_comments.reset_index(drop = True)

In [37]:
test_nyt = tfidfvect.transform(nyt_comments["commentBody"])

In [38]:
toxic = model[0].predict_proba(test_nyt)[:,1]
insult = model[3].predict_proba(test_nyt)[:,1]

In [39]:
np.where(insult > 0.95)[0]

array([ 260735,  558676,  629556,  664553,  729866, 1006160, 1396992,
       1487649, 1655902, 1676284, 2093533, 2093535])

In [40]:
nyt_comments.iloc[1006160]["commentBody"]

'well they were going to kill him anyway so well they were going to torture him anyway so well they were going to kill his whole family anyway so and so on'

In [47]:
result = nyt_comments[["commentID"]].copy()

In [48]:
result['toxic'] = model[0].predict_proba(test_nyt)[:,1]
result['severe_toxic'] = model[1].predict_proba(test_nyt)[:,1]
result['obscene'] = model[2].predict_proba(test_nyt)[:,1]
result['threat'] = model[3].predict_proba(test_nyt)[:,1]
result['insult'] = model[4].predict_proba(test_nyt)[:,1]
result['identity_hate'] = model[5].predict_proba(test_nyt)[:,1]

In [64]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [60]:
toxic_nyt_coms = nyt_comments.loc[np.where(result['toxic'] > .8)]["commentBody"].reset_index(drop = True)

In [74]:
x = [None]*len(toxic_nyt_coms)
for i in range(len(toxic_nyt_coms)):
     x[i] = sid.polarity_scores(toxic_nyt_coms[i])

In [79]:
toxic_comments = pd.DataFrame(toxic_nyt_coms)

In [81]:
toxic_comments["sent_anal"] = x

In [87]:
toxic_comments['neg'] = [d['neg'] for d in toxic_comments["sent_anal"]]

In [89]:
toxic_comments['neu'] = [d['neu'] for d in toxic_comments["sent_anal"]]

In [90]:
toxic_comments['pos'] = [d['pos'] for d in toxic_comments["sent_anal"]]

In [92]:
toxic_comments['compound'] = [d['compound'] for d in toxic_comments["sent_anal"]]

In [93]:
toxic_comments

,commentBody,sent_anal,neg,neu,pos,compound
0,i am waiting for people to stand up and scream...,"{'neg': 0.144, 'neu': 0.783, 'pos': 0.073, 'co...",0.144,0.783,0.073,-0.9261
1,trump is indeed a historic president he is the...,"{'neg': 0.421, 'neu': 0.377, 'pos': 0.202, 'co...",0.421,0.377,0.202,-0.8720
2,trump is a startlingly stupid man he is the so...,"{'neg': 0.448, 'neu': 0.552, 'pos': 0.0, 'comp...",0.448,0.552,0.000,-0.7964
3,disruption sucks,"{'neg': 1.0, 'neu': 0.0, 'pos': 0.0, 'compound...",1.000,0.000,0.000,-0.6124
4,this is not only an examination of a pathetic ...,"{'neg': 0.242, 'neu': 0.673, 'pos': 0.084, 'co...",0.242,0.673,0.084,-0.9041
5,so much for the christian right wing republica...,"{'neg': 0.199, 'neu': 0.668, 'pos': 0.133, 'co...",0.199,0.668,0.133,-0.8316
6,a policy that is cruel selfish and stupid utte...,"{'neg': 0.534, 'neu': 0.466, 'pos': 0.0, 'comp...",0.534,0.466,0.000,-0.8834
7,these are ugly killers who have little respect...,"{'neg': 0.258, 'neu': 0.678, 'pos': 0.064, 'co...",0.258,0.678,0.064,-0.9637
8,what garbage re writing history to sell books ...,"{'neg': 0.251, 'neu': 0.578, 'pos': 0.171, 'co...",0.251,0.578,0.171,-0.8183
9,the words are unimportant compared with some s...,"{'neg': 0.129, 'neu': 0.69, 'pos': 0.181, 'com...",0.129,0.690,0.181,0.6300
